In [24]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [25]:
# Step 1: Load the Data
vehicles_df = pd.read_csv('travels.csv')
reviews_df = pd.read_csv('reviews.csv')

In [26]:
vehicles_df.head()

,vehicle_id,travel_id,departure_at,from,to
0,65dda7755e43b9d7d3e4f3f9,65dda7765e43b9d7d3e4f48e,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
1,65dda7765e43b9d7d3e4f4bf,65dda7795e43b9d7d3e4f67d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
2,65dda7775e43b9d7d3e4f4e6,65dda77a5e43b9d7d3e4f6df,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
3,65dda7775e43b9d7d3e4f529,65dda77a5e43b9d7d3e4f74b,2024-03-03,"Pathri, Morong","Koteshowr, Kathmandu"
4,65dda7775e43b9d7d3e4f534,65dda77a5e43b9d7d3e4f753,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"


In [27]:
reviews_df.rename(columns={"id":'review_id'},inplace=True)

In [28]:
reviews_df.head()

,review_id,vehicle_id,user_id,rating
0,65dda7bea276ad551e8d07b2,65dda77d5e43b9d7d3e4f977,91,5
1,65dda7bea276ad551e8d07b3,65dda77f5e43b9d7d3e4fa4d,196,4
2,65dda7bea276ad551e8d07b4,65dda7855e43b9d7d3e4fe8c,180,3
3,65dda7bea276ad551e8d07b5,65dda7795e43b9d7d3e4f6a7,136,4
4,65dda7bea276ad551e8d07b6,65dda77c5e43b9d7d3e4f87f,141,3


In [29]:
print(reviews_df.shape)
print(vehicles_df.shape)

(120000, 4)
(103, 5)


In [30]:
# Step 2: Preprocess the Data
vehicles_with_reviews = pd.merge(reviews_df, vehicles_df, on='vehicle_id')

In [31]:
vehicles_with_reviews

,review_id,vehicle_id,user_id,rating,travel_id,departure_at,from,to
0,65dda7bea276ad551e8d07b2,65dda77d5e43b9d7d3e4f977,91,5,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
1,65dda7bea276ad551e8d07ff,65dda77d5e43b9d7d3e4f977,54,3,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
2,65dda7bea276ad551e8d0806,65dda77d5e43b9d7d3e4f977,154,5,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
3,65dda7bea276ad551e8d090c,65dda77d5e43b9d7d3e4f977,55,3,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
4,65dda7bea276ad551e8d0926,65dda77d5e43b9d7d3e4f977,192,5,65dda7815e43b9d7d3e4fbe6,2024-03-03,"Biratnagar, Sunsari","Kerkha, Jhapa"
...,...,...,...,...,...,...,...,...
119995,65dda7c7a276ad551e8ed8b7,65dda7835e43b9d7d3e4fd25,45,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119996,65dda7c7a276ad551e8ed9a8,65dda7835e43b9d7d3e4fd25,71,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119997,65dda7c7a276ad551e8ed9bb,65dda7835e43b9d7d3e4fd25,147,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119998,65dda7c7a276ad551e8edbcd,65dda7835e43b9d7d3e4fd25,2,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"


In [32]:
# Step 3: Filter Relevant Vehicles
def filter_vehicles(data, from_location, to_location, departure_at):
    filtered_data = data[(data['from'] == from_location) & (data['to'] == to_location) & (data['departure_at'] == departure_at)]
    return filtered_data

searched_vehicles = filter_vehicles(vehicles_with_reviews, "Pokhara, Kaski", "Pathri, Morong", "2024-03-03")

In [33]:
searched_vehicles

,review_id,vehicle_id,user_id,rating,travel_id,departure_at,from,to
1138,65dda7bea276ad551e8d07b3,65dda77f5e43b9d7d3e4fa4d,196,4,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1139,65dda7bea276ad551e8d07f3,65dda77f5e43b9d7d3e4fa4d,101,4,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1140,65dda7bea276ad551e8d0873,65dda77f5e43b9d7d3e4fa4d,181,3,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1141,65dda7bea276ad551e8d08ae,65dda77f5e43b9d7d3e4fa4d,104,5,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1142,65dda7bea276ad551e8d08c8,65dda77f5e43b9d7d3e4fa4d,31,5,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
...,...,...,...,...,...,...,...,...
119995,65dda7c7a276ad551e8ed8b7,65dda7835e43b9d7d3e4fd25,45,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119996,65dda7c7a276ad551e8ed9a8,65dda7835e43b9d7d3e4fd25,71,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119997,65dda7c7a276ad551e8ed9bb,65dda7835e43b9d7d3e4fd25,147,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119998,65dda7c7a276ad551e8edbcd,65dda7835e43b9d7d3e4fd25,2,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"


In [34]:
# Step 4: Generate Pivot table matrix
user_ratings = searched_vehicles.pivot_table(index='user_id', columns='vehicle_id', values='rating').fillna(0)

In [35]:
user_ratings

vehicle_id,65dda7765e43b9d7d3e4f4bf,65dda7775e43b9d7d3e4f4d3,65dda7775e43b9d7d3e4f54d,65dda7775e43b9d7d3e4f570,65dda7785e43b9d7d3e4f5cd,65dda7785e43b9d7d3e4f5e1,65dda7785e43b9d7d3e4f610,65dda7795e43b9d7d3e4f661,65dda77a5e43b9d7d3e4f712,65dda77b5e43b9d7d3e4f7aa,...,65dda77d5e43b9d7d3e4f952,65dda77e5e43b9d7d3e4f9e4,65dda77f5e43b9d7d3e4fa4d,65dda77f5e43b9d7d3e4fade,65dda7805e43b9d7d3e4fb2a,65dda7835e43b9d7d3e4fd25,65dda7845e43b9d7d3e4fd9d,65dda7845e43b9d7d3e4fdba,65dda7855e43b9d7d3e4fe52,65dda7865e43b9d7d3e4ff06
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.333333,3.500000,2.888889,4.000000,3.444444,3.250000,3.400000,3.200000,4.666667,3.000000,...,3.750000,3.250000,3.666667,3.750000,4.000000,3.200000,4.500000,3.666667,3.200000,3.500000
2,3.000000,3.444444,3.500000,4.500000,4.333333,3.000000,3.777778,3.000000,3.333333,2.600000,...,3.600000,3.375000,3.714286,4.000000,3.000000,3.888889,2.600000,4.000000,3.571429,3.545455
3,3.285714,3.666667,4.666667,3.400000,3.500000,4.250000,3.666667,3.428571,3.333333,4.100000,...,2.833333,3.750000,4.333333,3.666667,3.555556,3.250000,3.000000,3.000000,3.750000,4.000000
4,3.384615,3.833333,3.571429,3.166667,3.800000,3.500000,4.666667,3.250000,4.333333,3.428571,...,3.428571,4.000000,3.400000,3.500000,3.300000,3.000000,3.200000,3.750000,4.000000,3.666667
5,3.333333,4.285714,2.750000,2.500000,3.500000,3.333333,3.500000,4.571429,3.142857,4.000000,...,3.500000,3.000000,3.100000,4.000000,3.571429,3.888889,3.571429,2.666667,3.714286,3.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,3.250000,4.200000,2.500000,3.333333,3.666667,3.181818,4.000000,3.666667,3.000000,3.800000,...,3.000000,4.000000,3.285714,5.000000,3.538462,4.500000,3.400000,3.500000,4.333333,3.833333
197,2.600000,3.500000,3.777778,3.857143,3.600000,3.000000,2.888889,4.666667,3.333333,3.833333,...,3.000000,3.428571,2.000000,3.714286,3.000000,3.500000,3.571429,4.750000,2.000000,3.250000
198,3.000000,3.714286,3.285714,3.600000,3.200000,3.111111,3.000000,3.750000,3.444444,3.400000,...,3.400000,4.285714,3.444444,3.888889,2.400000,3.777778,4.000000,3.500000,4.333333,3.000000


In [36]:
# Step 4: Compute User to User Similarities
# Cosine similaritis over here will calculate over row wise means User to User 
similarity_matrix = cosine_similarity(user_ratings) 

# or if you wanted you can do Vehicles to Vehicles similarites for that you have to transpose
vehicles_similarity_matrix = cosine_similarity(user_ratings.T)

In [37]:
similarity_matrix.shape

(200, 200)

In [38]:
similarity_matrix

array([[1.        , 0.98044178, 0.97334315, ..., 0.98051135, 0.98928082,
        0.98372361],
       [0.98044178, 1.        , 0.9802626 , ..., 0.98340702, 0.97815627,
        0.98299543],
       [0.97334315, 0.9802626 , 1.        , ..., 0.98160994, 0.96968663,
        0.96721772],
       ...,
       [0.98051135, 0.98340702, 0.98160994, ..., 1.        , 0.98163671,
        0.98016723],
       [0.98928082, 0.97815627, 0.96968663, ..., 0.98163671, 1.        ,
        0.98417052],
       [0.98372361, 0.98299543, 0.96721772, ..., 0.98016723, 0.98417052,
        1.        ]])

In [41]:
vehicles_similarity_matrix.shape

(21, 21)

In [39]:
vehicles_similarity_matrix

array([[1.        , 0.97772211, 0.97705423, 0.97634276, 0.97248672,
        0.97837404, 0.97996631, 0.97873234, 0.97896604, 0.97512651,
        0.97663309, 0.98048483, 0.97761468, 0.97464267, 0.97875732,
        0.98050558, 0.97678153, 0.97634717, 0.98085989, 0.97905283,
        0.97933089],
       [0.97772211, 1.        , 0.97788219, 0.97601951, 0.97348358,
        0.9809686 , 0.98001432, 0.97565324, 0.97864343, 0.97454907,
        0.97734234, 0.97651259, 0.97862603, 0.97551494, 0.9817906 ,
        0.97866391, 0.98064912, 0.97866674, 0.97980006, 0.97830417,
        0.98040762],
       [0.97705423, 0.97788219, 1.        , 0.97613691, 0.97815845,
        0.97995313, 0.97978334, 0.97662673, 0.98159683, 0.97342473,
        0.97959672, 0.97473056, 0.98014144, 0.97922513, 0.97821308,
        0.98212413, 0.98104623, 0.97553742, 0.98146838, 0.97733542,
        0.97938019],
       [0.97634276, 0.97601951, 0.97613691, 1.        , 0.9737972 ,
        0.97815591, 0.97848104, 0.97801966, 0.9781269

In [46]:
# Creating dataframe out of numpy array from vehicles_similarity_matrix so that we can use mor easily
vehicles_similarity_matrix_df = pd.DataFrame(vehicles_similarity_matrix, index=user_ratings.columns, columns=user_ratings.columns)
vehicles_similarity_matrix_df
# Here '65dda7765e43b9d7d3e4f4bf' similar to '65dda7765e43b9d7d3e4f4bf' with 100%
# '65dda7765e43b9d7d3e4f4bf' similar to '65dda7775e43b9d7d3e4f4d3' with 97%
# Explanation: https://youtu.be/3ecNC-So0r4?t=1072

vehicle_id,65dda7765e43b9d7d3e4f4bf,65dda7775e43b9d7d3e4f4d3,65dda7775e43b9d7d3e4f54d,65dda7775e43b9d7d3e4f570,65dda7785e43b9d7d3e4f5cd,65dda7785e43b9d7d3e4f5e1,65dda7785e43b9d7d3e4f610,65dda7795e43b9d7d3e4f661,65dda77a5e43b9d7d3e4f712,65dda77b5e43b9d7d3e4f7aa,...,65dda77d5e43b9d7d3e4f952,65dda77e5e43b9d7d3e4f9e4,65dda77f5e43b9d7d3e4fa4d,65dda77f5e43b9d7d3e4fade,65dda7805e43b9d7d3e4fb2a,65dda7835e43b9d7d3e4fd25,65dda7845e43b9d7d3e4fd9d,65dda7845e43b9d7d3e4fdba,65dda7855e43b9d7d3e4fe52,65dda7865e43b9d7d3e4ff06
vehicle_id,,,,,,,,,,,,,,,,,,,,,
65dda7765e43b9d7d3e4f4bf,1.000000,0.977722,0.977054,0.976343,0.972487,0.978374,0.979966,0.978732,0.978966,0.975127,...,0.980485,0.977615,0.974643,0.978757,0.980506,0.976782,0.976347,0.980860,0.979053,0.979331
65dda7775e43b9d7d3e4f4d3,0.977722,1.000000,0.977882,0.976020,0.973484,0.980969,0.980014,0.975653,0.978643,0.974549,...,0.976513,0.978626,0.975515,0.981791,0.978664,0.980649,0.978667,0.979800,0.978304,0.980408
65dda7775e43b9d7d3e4f54d,0.977054,0.977882,1.000000,0.976137,0.978158,0.979953,0.979783,0.976627,0.981597,0.973425,...,0.974731,0.980141,0.979225,0.978213,0.982124,0.981046,0.975537,0.981468,0.977335,0.979380
65dda7775e43b9d7d3e4f570,0.976343,0.976020,0.976137,1.000000,0.973797,0.978156,0.978481,0.978020,0.978127,0.971687,...,0.978262,0.979747,0.973826,0.978598,0.979412,0.977309,0.975210,0.979204,0.974998,0.975540
65dda7785e43b9d7d3e4f5cd,0.972487,0.973484,0.978158,0.973797,1.000000,0.972169,0.978322,0.976005,0.980806,0.969443,...,0.975871,0.976098,0.973888,0.975526,0.974622,0.976424,0.976431,0.977107,0.974642,0.978695
65dda7785e43b9d7d3e4f5e1,0.978374,0.980969,0.979953,0.978156,0.972169,1.000000,0.979748,0.974663,0.977421,0.976826,...,0.971807,0.977648,0.975470,0.977727,0.979381,0.977526,0.974502,0.977606,0.976845,0.980287
65dda7785e43b9d7d3e4f610,0.979966,0.980014,0.979783,0.978481,0.978322,0.979748,1.000000,0.977896,0.982618,0.975203,...,0.976965,0.980819,0.977102,0.979338,0.979491,0.979098,0.979996,0.982314,0.981717,0.979714
65dda7795e43b9d7d3e4f661,0.978732,0.975653,0.976627,0.978020,0.976005,0.974663,0.977896,1.000000,0.979364,0.975997,...,0.978328,0.979471,0.977725,0.980268,0.978099,0.979043,0.977510,0.981238,0.977468,0.978383
65dda77a5e43b9d7d3e4f712,0.978966,0.978643,0.981597,0.978127,0.980806,0.977421,0.982618,0.979364,1.000000,0.975537,...,0.980131,0.982247,0.974696,0.980108,0.982011,0.978654,0.980176,0.980392,0.979403,0.981480


In [16]:
# NOTE: bellow here are all the code explanation that is done on 'get_recommendations' function
# filters the DataFrame searched_vehicles to get only the rows where the user_id matches the given user_id 8. It extracts all reviews made by the user.
user_id = 8
user_reviews = searched_vehicles[searched_vehicles['user_id'] == user_id]
user_reviews

,review_id,vehicle_id,user_id,rating,travel_id,departure_at,from,to
1150,65dda7bea276ad551e8d0b99,65dda77f5e43b9d7d3e4fa4d,8,3,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1304,65dda7bea276ad551e8d49d7,65dda77f5e43b9d7d3e4fa4d,8,3,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1326,65dda7bea276ad551e8d531d,65dda77f5e43b9d7d3e4fa4d,8,2,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
1977,65dda7c5a276ad551e8e658a,65dda77f5e43b9d7d3e4fa4d,8,2,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
2129,65dda7c5a276ad551e8ea126,65dda77f5e43b9d7d3e4fa4d,8,3,65dda7865e43b9d7d3e4ff0d,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
...,...,...,...,...,...,...,...,...
119070,65dda7bea276ad551e8d71d4,65dda7835e43b9d7d3e4fd25,8,3,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119127,65dda7c0a276ad551e8d899d,65dda7835e43b9d7d3e4fd25,8,5,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119320,65dda7c0a276ad551e8dd654,65dda7835e43b9d7d3e4fd25,8,2,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"
119392,65dda7c3a276ad551e8dedfc,65dda7835e43b9d7d3e4fd25,8,4,65dda7865e43b9d7d3e4ff17,2024-03-03,"Pokhara, Kaski","Pathri, Morong"


In [19]:
# This line creates a Pandas Series called user_ratings. 
# It extracts the rating values from the user_reviews DataFrame and sets the vehicle_id values as the index of the Series. 
# This Series represents the ratings given by the user to different vehicles.
user_ratings = pd.Series(user_reviews.rating.values, index=user_reviews.vehicle_id.values)
# here user could have rated the same vehicle multiple times so, we will going to take an average of that
user_ratings = user_reviews.groupby('vehicle_id')['rating'].mean()
user_ratings.shape

(21,)

In [20]:
user_ratings

vehicle_id
65dda7765e43b9d7d3e4f4bf    3.600000
65dda7775e43b9d7d3e4f4d3    3.166667
65dda7775e43b9d7d3e4f54d    3.083333
65dda7775e43b9d7d3e4f570    3.000000
65dda7785e43b9d7d3e4f5cd    3.444444
65dda7785e43b9d7d3e4f5e1    4.166667
65dda7785e43b9d7d3e4f610    3.444444
65dda7795e43b9d7d3e4f661    3.285714
65dda77a5e43b9d7d3e4f712    3.500000
65dda77b5e43b9d7d3e4f7aa    3.333333
65dda77c5e43b9d7d3e4f839    3.888889
65dda77d5e43b9d7d3e4f952    4.200000
65dda77e5e43b9d7d3e4f9e4    4.250000
65dda77f5e43b9d7d3e4fa4d    2.600000
65dda77f5e43b9d7d3e4fade    2.500000
65dda7805e43b9d7d3e4fb2a    3.500000
65dda7835e43b9d7d3e4fd25    3.400000
65dda7845e43b9d7d3e4fd9d    3.600000
65dda7845e43b9d7d3e4fdba    3.666667
65dda7855e43b9d7d3e4fe52    3.500000
65dda7865e43b9d7d3e4ff06    3.600000
Name: rating, dtype: float64

In [85]:
# These lines retrieve the similarity scores for the given user_id from the similarity_matrix. 
# Then, it sorts the similarity scores in descending order ([::-1]) and excludes the first element, which corresponds to the user itself. 
# This gives us an array of similar users sorted by similarity score.
similar_users = similarity_matrix[user_id]
similar_users = similar_users.argsort()[::-1][1:]  # and give the id of the user id
similar_users

array([112,  87,   7, 188,  60,  35, 118, 198, 158, 169, 189,  85, 147,
         0, 116, 121,  49,  65,  39, 132,  16,  94,  55,  27, 199, 148,
       160,  71,  31, 175, 149,  32,  23,  92,  67,  90, 182,  13,  86,
        28, 123,  48,  11,  74,  98,  14,  44, 165,  25,  76, 151, 184,
       133, 104, 126,  12, 102,  22, 163,  93,  69,  64, 113, 140,  75,
        73, 181, 172,  56, 162, 185, 130, 137,  54,  30, 183, 138,  36,
        37,  52, 110,  81, 103, 161, 154,  26,  51,  15,  46, 135, 139,
        20,   3, 120, 190, 155, 109,  72, 125, 108, 167, 197,  43, 128,
       124,  95,  99,  61,  88, 164, 105,  33, 178, 101,   6, 191,  59,
         4,   9,  78,  29, 193, 187, 141, 146, 127, 114, 170,  47, 106,
       166, 153, 194, 168,  57, 144,  89,  62,  83, 176,  10, 134, 152,
       142, 115,  41,  58,   1, 119, 180,  45, 196,   2,  96,  66, 159,
       117,  80, 157, 136,  17,  42,  82, 174, 177, 179, 129,  34,  21,
        97, 195, 192, 107, 150,  19,  77,  53,  40, 131, 145, 10

In [105]:
# Step 5: Generate Recommendations
def get_recommendations(user_id, searched_vehicles, similarity_matrix, n=5):
    user_reviews = searched_vehicles[searched_vehicles['user_id'] == user_id]
    user_ratings = pd.Series(user_reviews.rating.values, index=user_reviews.vehicle_id.values)
    # on index we are storing vehicle_id and on value we are storing user rating values
    
    similar_users = similarity_matrix[user_id]
    similar_users = similar_users.argsort()[::-1][1:]  # Exclude the user itself
    
    recommendations = defaultdict(float)
    for similar_user_id in similar_users:
        # retrieves single similar user reviews from 'searched_vehicles'
        similar_user_reviews = searched_vehicles[searched_vehicles['user_id'] == similar_user_id]
        for index, row in similar_user_reviews.iterrows():
            # now here check whether user have already rated a vehicle or not
            # row["vehicle_id"] (get vehicle id vehicle_id)
            # user_ratings[row['vehicle_id']] get info that whether user have rated this vehicle id or not
            # user_ratings[row['vehicle_id']].values == 0 (check whether user have rated vehicle_id as 0 or not)
            print(user_ratings[row['vehicle_id']].values.all() == 0 or row['vehicle_id'] not in user_ratings.index)
#             print(row['vehicle_id'] not in user_ratings.index)
#              if row['vehicle_id'] not in user_ratings.index or user_ratings[row['vehicle_id']].values == 0: 
                # Then it adds the weighted rating of the vehicle to the recommendations defaultdict.
#                  recommendations[row['vehicle_id']] += row['rating'] * similar_users[similar_user_id]
    # This line sorts the recommendations by their scores in descending order and selects the top n recommendations. 
    # It returns a list of tuples where each tuple contains the vehicle_id and its corresponding score.
    recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n]
    return recommendations

get_recommendations(user_id, searched_vehicles, similarity_matrix)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

...

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

[]

In [ ]:
# Function to recommend vehicles based on user input
def recommend_vehicles(from_location, to_location, departure_at, user_id, data):
    filtered_data = filter_vehicles(data, from_location, to_location, departure_at)
    user_similarity_matrix = compute_user_similarity(filtered_data)
    recommendations = get_recommendations(user_id, searched_vehicles, similarity_matrix)
    return recommendations